# Baseline_model.ipynb

In [2]:
import pandas as pd
import numpy as np
pd.set_option('precision', 5)

In [28]:
# load cleaned dataset
path = '/home/emilyboeke/'
master_app = pd.read_csv(path + 'merged_master_app.csv', low_memory=False)

In [29]:
# A BIT OF RESIDUAL DATA CLEANING (CAN BE OPTIMIZED/FIXED LATER)

# forgot to output excluding the auto-generated index column. dropping now
master_app = master_app.drop(master_app.columns[0], axis=1)

# drop NaNs. drops to 597954 obs
#tracid            598132
#nat               614012
#(other variables have 614204 observations)
master_app = master_app.dropna()

# change dec to binary number, so we can do summary stats on it
master_app.loc[(master_app["dec"] == 'DENY'),'dec'] = 0
master_app.loc[(master_app["dec"] == 'GRANT'),'dec'] = 1

master_app.describe()

,idnproceeding,idnProceedingAppln,idncase_x,numAppsPerProc,dec,idncase_y,tracid
count,5.97954e+05,5.97954e+05,5.97954e+05,597954.00000,597954.00000,5.97954e+05,597954.00000
mean,2.67211e+06,1.49888e+06,4.21033e+06,2.76359,0.35989,4.21033e+06,146.29203
std,1.47372e+06,1.48408e+06,1.24113e+06,0.78502,0.47997,1.24113e+06,85.11498
min,8.50000e+01,9.00000e+00,2.04693e+06,1.00000,0.00000,2.04693e+06,1.00000
25%,1.56721e+06,3.59521e+05,3.27020e+06,2.00000,0.00000,3.27020e+06,91.00000
50%,2.59977e+06,6.76532e+05,4.06603e+06,3.00000,0.00000,4.06603e+06,144.00000
75%,3.77913e+06,3.10490e+06,5.17135e+06,3.00000,1.00000,5.17136e+06,199.00000
max,6.18304e+06,4.67936e+06,7.20315e+06,14.00000,1.00000,7.20315e+06,403.00000


In [33]:
# selecting relevant features and target variable.
df = master_app[['osc_date', 'tracid', 'nat', 'dec']]
print(df.head(10))
df.describe()

      osc_date  tracid nat  dec
1   1994-11-04    71.0  HO    0
2   1994-11-05   139.0  GT    1
3   1994-11-05    70.0  ES    0
4   1995-02-06    50.0  HO    0
5   1995-02-06   126.0  CU    1
6   1995-02-10    71.0  NU    0
7   1995-02-10    71.0  NU    0
8   1995-02-12    61.0  NU    0
9   1995-02-17    70.0  NU    0
10  1995-02-17    70.0  NU    0


,tracid,dec
count,597954.00000,597954.00000
mean,146.29203,0.35989
std,85.11498,0.47997
min,1.00000,0.00000
25%,91.00000,0.00000
50%,144.00000,0.00000
75%,199.00000,1.00000
max,403.00000,1.00000


In [ ]:
# CHANGE OSC_DATE TO CONTINUOUS NUMBER. ORDER DATA BY DATE

# change osc_date to continuous number

# order data by date

In [94]:
# ONE HOT ENCODE CATEGORICAL VARIABLES

# change string nationalities to integer categories
from sklearn.preprocessing import LabelEncoder  
le = LabelEncoder()
nat_int = le.fit_transform(df['nat'])
nat_int = np.reshape(blah,[len(blah),1])

# get N x 2 array of features
feat_int = np.concatenate((df[['tracid']], nat_int), axis=1)
feat_int.shape

# get one hot encoder of features
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
enc.fit(feat_int)  

OneHotEncoder(categorical_features='all', dtype=<class 'numpy.float64'>,
       handle_unknown='error', n_values='auto', sparse=True)

In [ ]:
# DOES NOT WORK YET
# concatenate one hot encoded features with continuous features
features = np.concatenate((enc.transform(feat_int),np.reshape(df['osc_date'],[len(df),1])))

In [95]:
# SAVE THE LAST 20% OF DATA AND DO NOT TOUCH IT!

idx = round(feat_int.shape[0] * .8)
X = feat_int[0:idx]
y = df['dec'][0:idx]

X_test_donttouch = feat_int[idx:-1]
y_test_donttouch = df['dec'][idx:-1]


In [ ]:
# ridge regression


In [48]:
# walk forward validation